<a href="https://colab.research.google.com/github/sebastianiu/Prognosemodell_Online_Kreditzahlungsverkehr/blob/main/models/test/model_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Daten Laden

In [2]:
# Bibliotheken laden
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

url = "https://github.com/sebastianiu/Prognosemodell_Online_Kreditzahlungsverkehr/raw/main/data/raw/PSP_Jan_Feb_2019.xlsx"
Datensatz = pd.read_excel(url)
Datensatz = Datensatz.rename(columns = {"Unnamed: 0":"id"})

# 2. Datenaufbereitung & Bewertungsfunktionen

In [3]:
# Bilbiotheken laden
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import  precision_score,roc_curve,auc,accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, f1_score
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import cross_val_score

def Visualisierung_AUC(model,x_test,y_test):
  # Berechnung der Falsch-Positiv-Rate und der Wahr-Positiv-Rate für alle Schwellenwerte der Klassifizierung
  probs = model.predict_proba(x_test)
  preds = probs[:,1]
  fpr, tpr, threshold = roc_curve(y_test, preds)
  roc_auc = auc(fpr, tpr)
  # Visualisierung
  plt.title('Receiver Operating Characteristic')
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.legend(loc = 'lower right')
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show()

def Visualisierung_Class_Errors(model,x_test,y_test):
  #Vorhersage
  y_pred = model.predict(x_test)
  #Zippe Daten in Liste
  list(zip(y_pred,y_test))
  #Entzippe Liste
  unzip_file = [{'y_pred':y_pred,'y_test':y_test}for y_pred,y_test in zip(y_pred,y_test)]
  #Estelle DataFrame
  data = pd.DataFrame(unzip_file)
  #Ermittel Classification Error
  data['class_error'] = abs(data.y_pred-data.y_test)
  data = pd.concat([data,x_test], axis=1, join="inner")
  data['zaehler'] = 1

  # Decode Features
  data['PSP'] = label_encoder_PSP.inverse_transform(data['PSP'])
  data['country'] = label_encoder_country.inverse_transform(data['country'])
  data['card'] = label_encoder_card.inverse_transform(data['card'])
  data['weekday'] = label_encoder_weekday.inverse_transform(data['weekday'])

  max = data.amount.max()

  def create_amount_quantiles(row):
    if row['amount'] > 0 and row['amount'] <= max/4:
      result = '0 - '+str(max/4)
    else:
      if row['amount'] > max/4 and row['amount'] <= max/4*2:
        result = str(max/4+1)+' - '+str(max/4*2)
      else:
        if row['amount'] > max/4*2 and row['amount'] <= max/4*3:
          result = str(max/4*2+1)+' - '+str(max/4*3)
        else:
          result = str(max/4*3+1)+' - '+str(max)
    return result

  data['amount_quantiles'] = data.apply(create_amount_quantiles, axis=1)


  field_list = ['PSP','card','country','weekday','3D_secured','amount_quantiles']
  error_rates = pd.DataFrame(columns=['Merkmal','Merkmalswert','class_error','zaehler'])

  for field in field_list:
    errors = data.groupby(data[field])['class_error'].sum()
    total = data.groupby(data[field])['zaehler'].sum()
    error_rates_tmp = pd.concat([errors, total], axis=1)
    error_rates_tmp['Merkmalswert'] = error_rates_tmp.index.values
    error_rates_tmp['Merkmal']=field
    error_rates_tmp.reset_index()
    error_rates = pd.concat([error_rates,error_rates_tmp])

  error_rates['class_error_rate'] = error_rates.zaehler/error_rates.class_error
  max_error_rate = error_rates['class_error_rate'].max()

  # Visualisiere Verteilungen
  fig = px.bar(error_rates, x='Merkmalswert', y='class_error_rate', color= 'Merkmal',labels={'class_error_rate':'Klassifizierungsfehler in %'},title="Fehlerrate Klassifizierungen")
  fig.add_hline(y=max_error_rate,line_dash="dot",annotation_text=str(round(max_error_rate,2))+' %',annotation_position="top left")
  fig.show()

def Model_Bewertung(model,x_train, y_train,x_test, y_test,X,Y):
  y_pred_proba = model.predict_proba(x_test)
  cross_validation_tmp = cross_val_score(model, X, Y, cv=6)
  cross_validation=[]
  for value in cross_validation_tmp:
    new = round(value,3)
    cross_validation.append(new)


  print('Bewertungsmetriken')
  print('#'*20)
  print(f"Vorhersage-Genauigkeit auf Basis der Trainingsdaten: {round(model.score(x_train, y_train),2)}")
  print(f"Vorhersage-Genauigkeit auf Basis der Testdaten: {round(model.score(x_test, y_test),2)}")
  print('*'*15)
  precision_train = round(precision_score(y_train,model.predict(x_train),zero_division=1.0,average='weighted'),2)
  precision_test = round(precision_score(y_test,model.predict(x_test),zero_division=1.0,average='weighted'),2)
  print(f"Precision auf Basis der Trainingsdaten: {precision_train}")
  print(f"Precision auf Basis der Testdaten: {precision_test}")
  print('*'*15)
  f1_score_train = round(f1_score(y_train,model.predict(x_train),zero_division=1.0,average='weighted'),2)
  f1_score_test = round(f1_score(y_test,model.predict(x_test),zero_division=1.0,average='weighted'),2)
  print(f"F1-Score auf Basis der Trainingsdaten: {f1_score_train}")
  print(f"F1-Score auf Basis der Testdaten: {f1_score_test}")
  print('*'*15)
  print(f"Vorhersage-Genauigkeiten nach Kreuz-Validierung: {cross_validation}")
  print(f"Durschn. Vorhersage-Genauigkeit nach Kreuz-Validierung: {round(sum(cross_validation)/len(cross_validation),3)}")

  print('*'*15)

# Datenaufbereitung
label_encoder_PSP = LabelEncoder()
label_encoder_country = LabelEncoder()
label_encoder_card = LabelEncoder()
label_encoder_weekday = LabelEncoder()

ML_Daten = Datensatz.filter(['amount','success','PSP','country','card','tmsp','3D_secured'], axis=1)

ML_Daten['country'] = label_encoder_country.fit_transform(ML_Daten['country'])
ML_Daten['PSP'] = label_encoder_PSP.fit_transform(ML_Daten['PSP'])
ML_Daten['card'] = label_encoder_card.fit_transform(ML_Daten['card'])

# Datumswerte in Tag/ Wochentag/ Stunde aufteilen
ML_Daten['weekday'] = ML_Daten['tmsp'].dt.day_name()
ML_Daten['weekday'] = label_encoder_weekday.fit_transform(ML_Daten['weekday'])
ML_Daten['day'] = ML_Daten['tmsp'].dt.strftime('%d').astype(int)
ML_Daten['hour'] = ML_Daten['tmsp'].dt.strftime('%H').astype(int)

# Separation in X Merkmale and Zielvariable Y
Y = ML_Daten['success']
X = ML_Daten.filter(['amount','PSP','3D_secured','card','country','weekday','day','hour'], axis=1)

# Aufteilung in Trainings- und Validierungsdatensatz
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# 3. Bestes Modell feintunen

### 3.1 Merkmale reduzieren anhand Erkenntissen aus EDA und Separation in X Merkmale and Zielvariable Y

In [4]:
X2 = ML_Daten.filter(['amount','PSP','3D_secured'], axis=1)
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y, test_size=0.2)

## 3.2 Hyperparameter-Tuning mittels Kreuz-Validierung
#### 3.1 Random Forest Classifier
##### Hyperparameter-Raster

In [6]:
import numpy as np
# Hyper-Parameter für Raster
# Anzahl an Bäumen im RandomForest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Azahl der betrachteten Merkmale für jede Aufteilung
max_features = ['auto','sqrt']
# Maximale Anzahl an Ebenen in einem Baum
max_depth = [2,4]
# Minimale Anzahl der Proben, die zum Teilen eines Knotens erforderlich sind
min_samples_split = [2,5]
# Minimale Anzahl der Proben, die für jeden Knoten erforderlich sind
min_samples_leaf = [1,2]
#Bootstrap-Method zur Zusammenstellung der Trainingsdaten für jeden Baum verwenden - Ja/ Nein
bootstrap = [True, False]

# Erzeugen eines zufälligen Parameter-Rasters
param_grid= {'n_estimators': n_estimators,
            'max_features': max_features,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'bootstrap': bootstrap}

#### Umfassende Rastersuche

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rfc_final = RandomForestClassifier()
#rfc_final.fit(X2,Y)

rf_Gridsearch = GridSearchCV(estimator = rfc_final, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)
rf_Gridsearch.fit(x_train2,y_train2)

print('Optimale H-Parameter')
print('+'*20)
print(rf_Gridsearch.best_params_)

#Bewertung der ermittelten optimalen HP
print('Model-Bewertung mit Hyper-Parametern ermittelt mit GridSearchCV')
print(f'Vorhersage-Genauigkeit auf Basis der Trainingsdaten: {rf_Gridsearch.score(x_train2,y_train2):.3f}')
print(f'Vorhersage-Genauigkeit auf Basis der Testdaten: {rf_Gridsearch.score(x_test2,y_test2):.3f}')

Fitting 3 folds for each of 320 candidates, totalling 960 fits
Optimale H-Parameter
++++++++++++++++++++
{'bootstrap': True, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 33}
Model-Bewertung mit Hyper-Parametern ermittelt mit GridSearchCV
Vorhersage-Genauigkeit auf Basis der Trainingsdaten: 0.801
Vorhersage-Genauigkeit auf Basis der Testdaten: 0.797


#### Zufalls-basierte Parameter-Optimierung

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rf_GridsearchRandom = RandomizedSearchCV(estimator = rfc_final, param_distributions = param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
optimized_HParams = rf_GridsearchRandom.fit(x_train2,y_train2)

print('Optimale H-Parameter')
print('+'*20)
print(rf_GridsearchRandom.best_params_)

#Bewertung der ermittelten optimalen HP
print('Model-Bewertung mit Hyper-Parametern ermittelt mit RandomizedSearchCV')
print(f'Vorhersage-Genauigkeit auf Basis der Trainingsdaten: {rf_GridsearchRandom.score(x_train2,y_train2):.3f}')
print(f'Vorhersage-Genauigkeit auf Basis der Testdaten: {rf_GridsearchRandom.score(x_test2,y_test2):.3f}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.



Optimale H-Parameter
++++++++++++++++++++
{'n_estimators': 41, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 4, 'bootstrap': True}
Model-Bewertung mit Hyper-Parametern ermittelt mit RandomizedSearchCV
Vorhersage-Genauigkeit auf Basis der Trainingsdaten: 0.802
Vorhersage-Genauigkeit auf Basis der Testdaten: 0.793


RandomForestClassifier()

#### Regularisierung

#### Finales Training mit allen Daten

In [33]:
param_grid_tmp = rf_Gridsearch.best_params_
param_grid = pd.DataFrame(param_grid_tmp,index=[0])

rfc_final = RandomForestClassifier(
    bootstrap = param_grid.bootstrap.iloc[0]
    ,max_depth = param_grid.max_depth.iloc[0]
    ,max_features = param_grid.max_features.iloc[0]
    ,min_samples_leaf = param_grid.min_samples_leaf.iloc[0]
    ,min_samples_split = param_grid.min_samples_split.iloc[0]
    ,n_estimators = param_grid.n_estimators.iloc[0]
)

rfc_final.fit(X2,Y)

#Finale Bewertung
Model_Bewertung(rfc_final,x_train2, y_train2,x_test2, y_test2,X,Y)

Bewertungsmetriken
####################
Vorhersage-Genauigkeit auf Basis der Trainingsdaten: 0.8
Vorhersage-Genauigkeit auf Basis der Testdaten: 0.8
***************
Precision auf Basis der Trainingsdaten: 0.76
Precision auf Basis der Testdaten: 0.76
***************
F1-Score auf Basis der Trainingsdaten: 0.72
F1-Score auf Basis der Testdaten: 0.72
***************
Vorhersage-Genauigkeiten nach Kreuz-Validierung: [0.797, 0.797, 0.797, 0.797, 0.797, 0.797]
Durschn. Vorhersage-Genauigkeit nach Kreuz-Validierung: 0.797
***************
